# 単語埋め込み

ディープラーニングが自然言語処理に活用され始める前は、単語を計算可能なベクトルに変換する処理としては、文書集合内の各単語がある文書に出現した場合に1を、そうでない場合は0を出すようなBag of Wordsというものがよく用いられた。

ディープラーニングが活用されるようになり登場したものが単語の**埋め込み**（embedding）や**分散表現**（distributed representation）と呼ばれるもので、これらは$\{0, 1\}$に限らない値をとり、また次元数を語彙の数よりも小さく（次元圧縮）することもできる。

有名なのはWord2VecやElMoである。両者の大まかな違いとしては、Word2Vecは単語ごとに一意な分散表現になるが、ElMoは文脈によって分散表現が変わるということ。
例えば「彼はスポーツが下手だ」と「彼はいつも下手に出がちだ」は同じ「下手」という語だが意味が異なる。Word2Vecはこのような文脈を考慮しないが、ElMoは考慮する。

## 言語モデル

embeddingを直接推定するのではなく、単語予測モデル（言語モデル）を構築して副次的にembeddingを取得することになる。

言語モデルとは、尤もらしい文章を生成できるような確率分布を習得するために、文章の確率を推定するモデルのこと。

ある文章$S$をトークン化したのを$(w_1, w_2, \cdots, w_n)$と表記するならば、

$$
P(S)=P(w_1, w_2, \cdots, w_n)
$$

を求めたいということになる。これは条件付き確率の積として表せる

$$
\begin{align}
P(w_1, w_2, \cdots, w_n)
&= P(w_1) \times P(w_2|w_1) \times P(w_3|w_1, w_2) \times \cdots\\
&= \prod_{i=1}^n p(w_i|\boldsymbol{c}_i)
\end{align}
$$

ここで$\boldsymbol{c}_i$は$w_i$より前のトークン列$\boldsymbol{c}_i=(w_1,w_2,\cdots,w_n)$で、**文脈**（context）と呼ばれる



## Word2Vec

```
King - Man + Woman = Queen
```
のような語彙間の類似度や計算が可能な表現


### Word2Vecのアプローチは複数ある




Word2Vecの方法は、

1. CBOW (continuous bag of words)
2. Skip-Gram

の2種類あり、またそのままでは計算量が多すぎるために計算量削減の方法として

1. Hierarchical Softmax
2. Noise Constrastive Estimation
3. Negative Sampling

の方法があるため、組み合わせとしては2×3の6通りのモデルの組み方がある


### CBOW

**CBOW** (continuous bag of words)モデルは文章$S=(w_1, w_2, \dots, w_n)$が与えられた時、その$i$番目の単語$w_i$を、
その周りの単語である文脈$\boldsymbol{C}_i=(w_{i-c}, \dots, w_{i-1}, w_{i+1}, \dots, w_{i+c})$から予測するモデルである。ここで$c$はウィンドウサイズと呼ばれるハイパーパラメータ。







gensim

https://radimrehurek.com/gensim/models/word2vec.html

In [3]:
from gensim.test.utils import common_texts
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [8]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)

# 逐次学習
model.train([["hello", "world"]], total_examples=1, epochs=1)

(0, 2)

In [9]:
vector = model.wv['computer']  # get numpy vector of a word
vector

array([-0.00515774, -0.00667028, -0.0077791 ,  0.00831315, -0.00198292,
       -0.00685696, -0.0041556 ,  0.00514562, -0.00286997, -0.00375075,
        0.0016219 , -0.0027771 , -0.00158482,  0.0010748 , -0.00297881,
        0.00852176,  0.00391207, -0.00996176,  0.00626142, -0.00675622,
        0.00076966,  0.00440552, -0.00510486, -0.00211128,  0.00809783,
       -0.00424503, -0.00763848,  0.00926061, -0.00215612, -0.00472081,
        0.00857329,  0.00428459,  0.0043261 ,  0.00928722, -0.00845554,
        0.00525685,  0.00203994,  0.0041895 ,  0.00169839,  0.00446543,
        0.0044876 ,  0.0061063 , -0.00320303, -0.00457706, -0.00042664,
        0.00253447, -0.00326412,  0.00605948,  0.00415534,  0.00776685,
        0.00257002,  0.00811905, -0.00138761,  0.00808028,  0.0037181 ,
       -0.00804967, -0.00393476, -0.0024726 ,  0.00489447, -0.00087241,
       -0.00283173,  0.00783599,  0.00932561, -0.0016154 , -0.00516075,
       -0.00470313, -0.00484746, -0.00960562,  0.00137242, -0.00

In [7]:
sims = model.wv.most_similar('computer', topn=10)  # get other similar words
sims

[('system', 0.21617139875888824),
 ('survey', 0.04468922317028046),
 ('interface', 0.015203381888568401),
 ('time', 0.0019510635174810886),
 ('trees', -0.03284316882491112),
 ('human', -0.07424270361661911),
 ('response', -0.09317591041326523),
 ('graph', -0.09575342386960983),
 ('eps', -0.10513808578252792),
 ('user', -0.16911619901657104)]

### 参考文献

- https://code.google.com/archive/p/word2vec/
- [[1310.4546] Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)